In [1]:
import pandas;
import seaborn as sb;
import numpy as np;
%matplotlib inline
np.set_printoptions(suppress=True)

In [178]:
def readCSVFile(file):
    data=pandas.read_csv(file,",",header=0, na_values='?', skipinitialspace=True);
    return data;
    pass;
def readTrainData(dataset):    
    return dataset.ix[:,1:-1], dataset.ix[:,-1:];
    pass;

def readTestData(dataset):    
    return dataset.ix[:,1:],dataset.ix[:,0:1];
    pass;

def normalizePhi(unNormalizedPhi,last_col_bias=False):    
    #assuming last column as bias column
    no_of_column=len(unNormalizedPhi[0]);
    phi=np.array(unNormalizedPhi);
    print("Normalizing Phi...");  
    std=phi.std(0);
    mean=phi.mean(0);    
    std[no_of_column-1]=1;
    mean[no_of_column-1]=0;
    #phi_normalize=(phi-mean)/std;    
    
    max_vector=phi.max(axis=0)
    phi_normalize=phi/max_vector;    
    
    print("Normalization done.");
    return phi_normalize;
    pass;

def writeTestData(test_id,ystar,filenumber=0):
    if filenumber>0:
        fo = open("sampleSubmission-"+str(filenumber)+".csv", "w");    
    else:
        fo = open("sampleSubmission.csv", "w");        
    fo.write("id,salary\n");
    m=len(ystar);
    for i in range(m):
        fo.write(str(test_id[i][0])+","+str(ystar[i][0])+"\n");
    fo.close();
    pass;

def dropColumns(dataframe):
    dataframe.drop(['native-country'], axis = 1, inplace = True);
    pass;

def addColByCategory(dataset):
    return pandas.get_dummies(dataset);
    pass;

def categoryToNumber(dataset,categoryList):
    for c in categoryList:
        if (c in dataset):            
            dataset[c]=pandas.get_dummies(dataset[c]).values.argmax(1);        
    return dataset;
    pass;
    

def handleCategoryData(dataset,categoryList=None,byNumber=False):
    if(byNumber):
        return categoryToNumber(dataset,categoryList)
    else:
        return addColByCategory(dataset);
#b=

In [180]:
#--settings--
pandas.set_option('display.max_columns', None);
#---init---
dir="data/"
trainFile=dir+"train.csv";
testFile=dir+"kaggle_test_data.csv";
categoryList=["workclass","education","marital-status","occupation","relationship","race","sex","native-country"];
trained_dataset=readCSVFile(trainFile);
trained_data,trained_y=readTrainData(trained_dataset);

test_dataset=trained_dataset=readCSVFile(testFile);
test_data,test_id=readTestData(test_dataset);

#droping unrelated-columns
#dropColumns(trained_data);
#dropColumns(test_data);

#converting categorical data to point wise data
byNumber=True;
dummy_trained_data=handleCategoryData(trained_data,categoryList,byNumber);
dummy_test_data=handleCategoryData(test_data,categoryList,byNumber);

#trained_columns_name=list(dummy_trained_data.columns.values);
#test_columns_name=list(dummy_test_data.columns.values);

#converting panda data frame to numpy martix
mtx_dummy_tds=dummy_trained_data.as_matrix(columns=None)
mtx_dummy_testds=dummy_test_data.as_matrix(columns=None)
mtx_trained_y=trained_y.as_matrix(columns=None);
mtx_test_id=test_id.as_matrix(columns=None);

#adding bias column
mtx_dummy_tds=np.column_stack((mtx_dummy_tds,np.ones((len(mtx_dummy_tds),1))))
mtx_dummy_testds=np.column_stack((mtx_dummy_testds,np.ones((len(mtx_dummy_testds),1))))

#normalization
mtx_dummy_tds_norm=normalizePhi(mtx_dummy_tds)
mtx_dummy_testds_norm=normalizePhi(mtx_dummy_testds)

#print(mtx_dummy_tds)
#print(mtx_dummy_tds_norm)
#pandas.get_dummies(trained_data.ix[:,1:2])
print("train",np.shape(mtx_dummy_tds_norm),"test",np.shape(mtx_dummy_testds_norm))


Normalizing Phi...
Normalization done.
Normalizing Phi...
Normalization done.
[[     25.       3.  226802. ...,      40.      38.       1.]
 [     38.       3.   89814. ...,      50.      38.       1.]
 [     28.       1.  336951. ...,      40.      38.       1.]
 ..., 
 [     22.       3.  310152. ...,      40.      38.       1.]
 [     58.       3.  151910. ...,      40.      38.       1.]
 [     22.       3.  201490. ...,      20.      38.       1.]]
[[ 0.27777778  0.42857143  0.15217525 ...,  0.4040404   0.95        1.        ]
 [ 0.42222222  0.42857143  0.06026167 ...,  0.50505051  0.95        1.        ]
 [ 0.31111111  0.14285714  0.22608092 ...,  0.4040404   0.95        1.        ]
 ..., 
 [ 0.24444444  0.42857143  0.20809984 ...,  0.4040404   0.95        1.        ]
 [ 0.64444444  0.42857143  0.10192566 ...,  0.4040404   0.95        1.        ]
 [ 0.24444444  0.42857143  0.13519189 ...,  0.2020202   0.95        1.        ]]
train (38973, 15) test (6878, 15)


In [72]:
pandas.get_dummies(trained_dataset)

,id,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,salary,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_10th,education_11th,education_12th,education_1st-4th,education_5th-6th,education_7th-8th,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Preschool,education_Prof-school,education_Some-college,marital-status_Divorced,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,occupation_Adm-clerical,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White,sex_Female,sex_Male,native-country_Cambodia,native-country_Canada,native-country_China,native-country_Columbia,native-country_Cuba,native-country_Dominican-Republic,native-country_Ecuador,native-country_El-Salvador,native-country_England,native-country_France,native-country_Germany,native-country_Greece,native-country_Guatemala,native-country_Haiti,native-country_Holand-Netherlands,native-country_Honduras,native-country_Hong,native-country_Hungary,native-country_India,native-country_Iran,native-country_Ireland,native-country_Italy,native-country_Jamaica,native-country_Japan,native-country_Laos,native-country_Mexico,native-country_Nicaragua,native-country_Outlying-US(Guam-USVI-etc),native-country_Peru,native-country_Philippines,native-country_Poland,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,0,25,226802,7,0,0,40,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1,38,89814,9,0,0,50,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2,28,336951,12,0,0,40,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,3,44,160323,10,7688,0,40,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [68]:
pandas.read_csv(filepath_or_buffer=trainFile, sep=',', header=0, skipinitialspace=True, na_values='?')

,id,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,0
1,1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,0
2,2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,1
3,3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,1
4,5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,0
5,7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103,0,32,United-States,1
6,8,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0,0,40,United-States,0
7,9,55,Private,104996,7th-8th,4,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,10,United-States,0
8,10,65,Private,184454,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,6418,0,40,United-States,1
9,12,26,Private,82091,HS-grad,9,Never-married,Adm-clerical,Not-in-family,White,Female,0,0,39,United-States,0


In [108]:
def singleLayerTestNN():  
    # sigmoid function
    def nonlin(x,deriv=False):
        if(deriv==True):
            return x*(1-x)
        return 1/(1+np.exp(-x))

    # input dataset
    X = np.array([  [0,0,1],
                    [0,1,1],
                    [1,0,1],
                    [1,1,1] ])

    # output dataset            
    y = np.array([[0,0,1,1]]).T

    # seed random numbers to make calculation
    # deterministic (just a good practice)
    np.random.seed(1)

    # initialize weights randomly with mean 0
    syn0 = 2*np.random.random((3,1)) - 1
    print("W1:",syn0);
    for iter in range(10000):
        
        # forward propagation
        l0 = X
        l1 = nonlin(np.dot(l0,syn0))
        #print("L1:",l1);
        # how much did we miss?
        l1_error = y - l1
        #print("L1Error:",l1_error);
        if (iter% 10000) == 0:
            print("Error:" + str(np.mean(np.abs(l1_error))))
            
        # multiply how much we missed by the 
        # slope of the sigmoid at the values in l1
        l1_delta = l1_error * nonlin(l1,True)
        #print("L1Delta_partial:",l1_delta);        
        #print("L1Delta:",np.dot(l0.T,l1_delta));
        
        # update weights
        syn0 += np.dot(l0.T,l1_delta)
        
    print("Output After Training:");
    print(syn0);

singleLayerTestNN()

W1: [[-0.16595599]
 [ 0.44064899]
 [-0.99977125]]
Error:0.517208275438
Output After Training:
[[ 9.67299303]
 [-0.2078435 ]
 [-4.62963669]]


In [81]:
#http://iamtrask.github.io/2015/07/12/basic-python-network/
def batchSingleHiddenLayerTestNN():
    def nonlin(x,deriv=False):
        if(deriv==True):
            return x*(1-x)
        return 1/(1+np.exp(-x))

    X = np.array([[0,0,1],
                [0,1,1],
                [1,0,1],
                [1,1,1]])

    y = np.array([[0],[1],[1],[0]])

    np.random.seed(1)

    # randomly initialize our weights with mean 0
    syn0 = 2*np.random.random((3,4)) - 1
    syn1 = 2*np.random.random((4,1)) - 1  
    syn1=np.array(syn1);
    print("syn0",syn0);
    print("syn1",syn1);
    n=60000
    for j in range(n):

        # Feed forward through layers 0, 1, and 2
        l0 = X
        l1 = nonlin(np.dot(l0,syn0))
        #print("l1:",l1);
        l2 = nonlin(np.dot(l1,syn1))
        #print("l2:",l2);
        
        # how much did we miss the target value?
        l2_error = y - l2
        if(j==n-1):
                print("X:",l0," PY:",l2, " AY:",y);
        if (j% 10000) == 0:
            print("Error:" + str(np.mean(np.abs(l2_error))))

        # in what direction is the target value?
        # were we really sure? if so, don't change too much.
        l2_delta = l2_error*nonlin(l2,deriv=True)
        #print("L2delta:",l2_delta)

        
        # how much did each l1 value contribute to the l2 error (according to the weights)?
        l1_error = l2_delta.dot(syn1.T)
        #print("L1Error:",l1_error)

        # in what direction is the target l1?
        # were we really sure? if so, don't change too much.
        l1_delta = l1_error * nonlin(l1,deriv=True)
        #print("L1delta:",l1_delta)

        syn1 += l1.T.dot(l2_delta)
        syn0 += l0.T.dot(l1_delta)
        #print("syn1",syn1);
        #print("syn0",syn0);
batchSingleHiddenLayerTestNN();

syn0 [[-0.16595599  0.44064899 -0.99977125 -0.39533485]
 [-0.70648822 -0.81532281 -0.62747958 -0.30887855]
 [-0.20646505  0.07763347 -0.16161097  0.370439  ]]
syn1 [[-0.5910955 ]
 [ 0.75623487]
 [-0.94522481]
 [ 0.34093502]]
Error:0.496410031903
Error:0.00858452565325
Error:0.00578945986251
Error:0.00462917677677
Error:0.00395876528027
Error:0.00351012256786
X: [[0 0 1]
 [0 1 1]
 [1 0 1]
 [1 1 1]]  PY: [[ 0.00260572]
 [ 0.99672209]
 [ 0.99701711]
 [ 0.00386759]]  AY: [[0]
 [1]
 [1]
 [0]]


In [45]:
def singleHiddenLayerTestNN():
    def nonlin(x,deriv=False):
        if(deriv==True):
            return x*(1-x)
        return 1/(1+np.exp(-x))

    X = np.array([[0,0,1],
                [0,1,1],
                [1,0,1],
                [1,1,1]])
    y = [0,1,1,0]
    np.random.seed(1)

    # randomly initialize our weights with mean 0
    syn0 = 2*np.random.random((3,4)) - 1
    syn1 = 2*np.random.random((4)) - 1    
    dataset_size=len(X);
    print("sys0",syn0);
    print("syn1",syn1);
    n=60000;
    for j in range(n):        
        total_error=list();
        for i in range(dataset_size):
            # Feed forward through layers 0, 1, and 2
            #print("i=",i);
            yi=y[i];
            l0 = X[i];               
            #print("l0",l0);
            l1 = nonlin(np.dot(l0.T,syn0))
            #print("l1",np.dot(l0.T,syn0));            
            l2 = nonlin(np.dot(l1.T,syn1));
            #print("l2",l2);
            if(j==n-1):
                print("X:",l0," PY:",l2, " AY:",yi);
                print("syn1",syn1);
                print("syn0",syn0);
            
            # how much did we miss the target value?
            l2_error = yi - l2   
            total_error.append(l2_error);
            #if(l2_error==0):#point is not misclassified
            #   continue;        
            # in what direction is the target value?
            # were we really sure? if so, don't change too much.
            l2_delta = l2_error*nonlin(l2,deriv=True)
            #print("L2delta:",l2_delta)


            # how much did each l1 value contribute to the l2 error (according to the weights)?
            l1_error = l2 * syn1
            #print("L1Error:",l1_error)
            #print("non",nonlin(l1,deriv=True));
            # in what direction is the target l1?
            # were we really sure? if so, don't change too much.
            l1_delta = l1_error * np.array(nonlin(l1,deriv=True))
            #print("l1_delta:",l1_delta);
            
            syn1 += l1.T*l2_delta;
            syn0 += l0.reshape(len(l0),1).dot(l1_delta.reshape(1,len(l1_delta)))
            #print("syn1",syn1);
            #print("syn0",syn0);
            
        #end of loop inner loop
        if  (j% 10000) == 0 and len(total_error)>0:
                print("Error:" + str(np.mean(np.abs(total_error))))
    pass;
        
#singleHiddenLayerTestNN();

In [44]:
X = np.array([[0,0,1],
                [0,1,1],
                [1,0,1],
                [1,1,1]])


len(X)
a=list();
a.append(1);
np.mean(np.abs([1,2,3,4,-5]))
np.array([1,2,3,4,-5])
np.array([1,2,3,4,-5]) * np.array([1,2,3,4,-5])
b=np.array([1,2,3,4,-5]).reshape((5, 1));
a=np.dot(b,np.array([1,2,3,4,-5]).reshape((1, 5)))


In [120]:
class NN:
    logging_enabled=True;
    trained_ds=None;
    trained_output=None;
    no_of_hiddenlayer=0;
    neurons_per_hiddenlayer=0;    
    weights=list();#list of matrix of weights
    layer_output=list();#vector of layers
    layer_delta=list(); 
    layer_error=list(); 
    no_of_iteration=1;
    mean_print_rate=10000;
    percent_print_rate=1000;
    learning_rate=1;
    approching_to_zero=1e-15;
    approching_to_one=1-1e-15;
    lamda=0.001;
    log_dir="log";
    log_folder=None;
    def __init__(self):
        np.random.seed(1);
        self.log_folder=self.log_dir+"/"+self.getTimestamp();
        self.createDir(self.log_folder);
        pass;
    
    def reInit(self):        
        self.layer_output=[0]*(self.no_of_hiddenlayer+2);
        self.layer_delta=[0]*(self.no_of_hiddenlayer+2);
        self.layer_error=[0]*(self.no_of_hiddenlayer+2);        
        pass;
        
    def createNN(self):
        self.no_of_features=len(self.trained_ds[0]);
        self.no_of_datapoint=len(self.trained_ds); 
        self.reInit();
        self.initWeightMatrix();       
        print("Neural Network Created...");
        pass;
    
    def initWeightMatrix(self):                
        for i in range(self.no_of_hiddenlayer):
            if(i==0):
                m=self.no_of_features;
            else:
                m=self.neurons_per_hiddenlayer;
            w_matrix = 2*np.random.random((m,self.neurons_per_hiddenlayer)) - 1;            
            self.weights.append(w_matrix);
       
        #last layer weight: For single output
        if(self.no_of_hiddenlayer==0):
            n=self.no_of_features;
        else:
            n=self.neurons_per_hiddenlayer;
        w_vector=2*np.random.random((n,1)) - 1; 
        self.weights.append(w_vector);
        pass;
        
    def getTimestamp(self):
        import os.path;
        import datetime;
        import time;
        ts = datetime.datetime.fromtimestamp(time.time()).strftime('%d-%m-%Y-%H:%M:%S')
        return ts;

    def createDir(self,directory):
        import os.path;
        if not os.path.exists(directory):
            os.makedirs(directory);
        pass;
    
    def writeWeights(self,filenumber=0):
        fo = open(self.log_folder+"/weights-"+str(filenumber)+".csv", "w");    
        m=len(self.weights);
        for i in range(1):
            print(self.weights[i])
            fo.write("w"+str(i)+")------------------\n");            
            np.savetxt(fo,self.weights[i],fmt=format);
            #np.savetxt(fo,np.array(self.weights[i]));            
            fo.write("\n--------------------------\n");
        fo.close();
        pass;
    
    def activationFunction(self,x,deriv=False):
        return self.actFunSigmoid(x,deriv);
    
    def actFunSigmoid(self,x,deriv=False):
        if(deriv==True):
            return x*(1-x)
        return 1/(1+np.exp(-x))
    
    def actFunReLu(self,x,deriv=False):
        p = x > 0;            
        if(deriv==True):
            return p.astype(int);
        return x * p;
    
    def forwardPropogation(self,datapoints):        
        #intial layer i.e l0 zeroth layer
        self.layer_output[0]=datapoints;
        for i in range(1,self.no_of_hiddenlayer+2):
            prev_layer=self.layer_output[i-1];
            w=self.weights[i-1]
            sum_l=prev_layer.dot(w);           
            l_i=self.activationFunction(sum_l);
            self.layer_output[i]=l_i;           
        return self.layer_output[self.no_of_hiddenlayer+1];
        pass;
    
    
    def backPropogation(self,l_output):
        last_layer=len(l_output)-1;            
        
        #(target-output)        
        li=l_output[last_layer];
        error_diff=self.trained_output-li;
        delta=error_diff*self.activationFunction(li,deriv=True)    
        self.layer_delta[last_layer]=delta;
        self.layer_error[last_layer]=error_diff;
        
        for i in range(last_layer-1,0,-1):            
            #i-1 th layer calculation of delta
            error_diff=self.layer_delta[i+1].dot(self.weights[i].T);
            #print("l"+str(i)+"_error:",delta);
            delta=error_diff*self.activationFunction(l_output[i],deriv=True)
            self.layer_delta[i]=delta;
            self.layer_error[i]=error_diff;  
            
        #print("l"+str(i)+"_delta:",delta);        
        pass;
    
    def updateWeights(self,l_delta):
        #print(self.learning_rate);
        for i in range(self.no_of_hiddenlayer,-1,-1): # loop upto 0
            #print(str(i)+"==>a",self.weights[i][0]);            
            #print(str(i)+"==>b",self.layer_output[i].T.dot(l_delta[i+1])[0])
            #print(str(i)+"==>c",(self.lamda*self.weights[i])[0])            
            self.weights[i]+=self.learning_rate*(self.layer_output[i].T.dot(l_delta[i+1])-((self.lamda)*self.weights[i]));
            #print("w"+str(i)+":",self.layer_output[i].T.dot(l_delta[i+1]));
    
    
    def getMeanError(self):   
        return np.mean(np.abs(self.layer_error[self.no_of_hiddenlayer+1]));
    
    
    def gradientDescent(self):
        for i in range(self.no_of_iteration):
            self.forwardPropogation(self.trained_ds);
            self.backPropogation(self.layer_output);                       
            if (i%self.mean_print_rate==0 and self.logging_enabled):
                       print("Error:",self.getMeanError()); 
            if (i%self.percent_print_rate==0):
                self.log("Percent completed",(i*100)/self.no_of_iteration)
                #self.writeWeights();                
            self.updateWeights(self.layer_delta);
        print("Error:",self.getMeanError());
        pass;
    
    def train(self):
         self.gradientDescent();
            
    def log(self,text,data):
        if self.logging_enabled:
            print(text,data);
        pass;
pass;
nn= NN();
x = np.array([[0,0,1],
                [0,1,1],
                [1,0,1],
                [1,1,1]])

y = np.array([[0],[1],[1],[1]])

x=mtx_dummy_tds_norm;
y=mtx_trained_y;

#dataset OR
#nn.no_of_hiddenlayer=1;
#nn.neurons_per_hiddenlayer=4
#nn.no_of_iteration=1000;
#nn.learning_rate=1;
#nn.lamda=0;
#nn.mean_print_rate=100;
#nn.percent_print_rate=100;

#dataset: census income
nn.no_of_hiddenlayer=2;
nn.neurons_per_hiddenlayer=120
nn.no_of_iteration=10000;
nn.learning_rate=0.000001;
nn.lamda=2;
nn.mean_print_rate=20;
nn.percent_print_rate=20;

#Common setting
nn.logging_enabled=True;
nn.trained_ds=x;
nn.trained_output=y;
nn.createNN();
nn.train();
result=nn.forwardPropogation(x);
print("predicted",result);
print("Actual",y);
result=nn.forwardPropogation(mtx_dummy_testds_norm);
writeTestData(mtx_test_id,result);


Neural Network Created...
Error: 0.687380307041
Percent completed 0.0
Error: 0.442866986606
Percent completed 0.2
Error: 0.367089069466
Percent completed 0.4
Error: 0.34445574484
Percent completed 0.6
Error: 0.335952690831
Percent completed 0.8
Error: 0.333192206711
Percent completed 1.0
Error: 0.333305715353
Percent completed 1.2
Error: 0.334898690894
Percent completed 1.4
Error: 0.337083715708
Percent completed 1.6
Error: 0.339233061009
Percent completed 1.8
Error: 0.340942904287
Percent completed 2.0
Error: 0.342025108817
Percent completed 2.2
Error: 0.342460751788
Percent completed 2.4
Error: 0.342327857458
Percent completed 2.6
Error: 0.341738697309
Percent completed 2.8
Error: 0.340803247475
Percent completed 3.0
Error: 0.339614773522
Percent completed 3.2
Error: 0.33824744041
Percent completed 3.4
Error: 0.336758534857
Percent completed 3.6
Error: 0.335191709731
Percent completed 3.8
Error: 0.333579972183
Percent completed 4.0
Error: 0.331948119835
Percent completed 4.2
Error: 0

In [119]:
approching_to_zero=1e-15
approching_to_one=1-1e-15
a=np.array([[0,0.345,1],[0.11,0.000000002,0.99999]])

p=(a == 0).astype(int)
c = np.empty((2,3))
c.fill(approching_to_zero);
#result=nn.forwardPropogation(mtx_dummy_testds_norm);
#print("predicted",result);
#print("Actual",y);
writeTestData(mtx_test_id,result);

In [111]:
p=(result>0.49).astype(int)
print(np.abs(y-p).sum())
for i in range(100):
    #print(result[i],":",p[i],":",y[i])

In [109]:
a=np.array([[1,2],[3.345345345443,6]])
np.savetxt('exampleData.csv', a, delimiter=',')
